In [1]:
import pandas as pd
import geopandas as gpd
import pickle
from shapely.geometry import Polygon
import time

%matplotlib inline
import matplotlib.pyplot as plt

root_path = "D:/GeoData/"
Main_CRS = "EPSG:27700"

In [2]:
#Function for opening saved data
def load_obj(name ):
    with open(root_path + 'WorkingData/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [3]:
All_Maps = load_obj("All_Maps")

Clipping functions

In [4]:
def MakePolygon(centre, sq_sz, inp_crs):
    mnx = centre[0]-(sq_sz/2)
    mny = centre[1]-(sq_sz/2)
    mxx = centre[0]+(sq_sz/2)
    mxy = centre[1]+(sq_sz/2)
    polygon =Polygon([(mnx, mny), (mnx,mxy), (mxx,mxy), (mxx,mny), (mnx, mny)])
    poly_gdf = gpd.GeoDataFrame([1], geometry=[polygon], crs=inp_crs)
    return poly_gdf

In [10]:
def clip_map(source_gdf, clip_gdf, geometry = 'geometry'):
    #extract the geometry of the area you want to clip to
    clip_geom = clip_gdf[geometry].iloc[0]

    #use an r-tree to find the intersection
    spatial_index = source_gdf.sindex
    possible_matches_index  = list(spatial_index.intersection(clip_geom.bounds))
    possible_matches = source_gdf.iloc[possible_matches_index]
    precise_matches = possible_matches[possible_matches.intersects(clip_geom)]
    clipped = gpd.clip(source_gdf, clip_gdf)
    return clipped

In [6]:
Clip_poly = MakePolygon([535493, 171250], 5000, Main_CRS)

In [8]:
#Original Method
start = time.time()

maps = dict((k, gpd.clip(v, Clip_poly)) for k, v in All_Maps.items())

end = time.time()
clip_method_time = end - start

In [11]:
#New Method
start = time.time()

maps = dict((k, clip_map(v, Clip_poly)) for k, v in All_Maps.items())

end = time.time()
r_tree_method_time = end - start

In [12]:
print("Old time is", clip_method_time, "New time is", r_tree_method_time)

Old time is 909.1514949798584 New time is 1413.0587012767792
